In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [22]:
df = pd.read_csv('../data/occupation_shares.csv')
df = df.iloc[:-1]
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0)
df = df.set_index('MET2013')

In [23]:
def get_shifter(pi_c, pi_c_prime, rho_k=0.5):
    return (pi_c / pi_c_prime)**(1 - rho_k)

# Assuming df is your DataFrame
df_new = df.copy()
df_new = df_new.iloc[1:]

for column in df_new.columns:
    pi_c_prime = df_new.iloc[0][column]
    df_new[column] = df_new[column].apply(lambda x: get_shifter(x, pi_c_prime))
df_new.to_csv('shifter_estimates.csv')